In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
187,AFG,Asia,Afghanistan,2020-07-15,34740.0,285.0,1045.0,33.0,892.409,7.321,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
316,ALB,Europe,Albania,2020-07-15,3667.0,96.0,97.0,2.0,1274.237,33.359,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
509,DZA,Africa,Algeria,2020-07-15,20216.0,527.0,1028.0,10.0,461.015,12.018,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-14,France,172583.0,0.0,30036.0,0.0,0.0,0.0
0,2020-07-14,China,85244.0,0.0,4651.0,0.0,0.0,0.0
0,2020-07-14,Italy,243589.0,0.0,35000.0,0.0,0.0,0.0
0,2020-07-14,Spain,256020.0,0.0,28419.0,0.0,0.0,0.0
0,2020-07-14,United States,3417102.0,0.0,136223.0,0.0,0.0,0.0
0,2020-07-14,World,13352870.0,0.0,578176.0,0.0,0.0,0.0
0,2020-07-14,United Kingdom,291107.0,0.0,44915.0,0.0,0.0,0.0
0,2020-07-14,Germany,199235.0,0.0,9074.0,0.0,0.0,0.0
0,2020-07-14,Iran,262964.0,0.0,13200.0,0.0,0.0,0.0
0,2020-07-14,Turkey,214966.0,0.0,5400.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-15,France,172583.0,172377.0,30036.0,30029.0,206.0,7.0
0,2020-07-15,China,85244.0,85226.0,4651.0,4642.0,18.0,9.0
0,2020-07-15,Italy,243589.0,243344.0,35000.0,34984.0,245.0,16.0
0,2020-07-15,Spain,256020.0,256619.0,28419.0,28409.0,-599.0,10.0
0,2020-07-15,United States,3417102.0,3431574.0,136223.0,136466.0,-14472.0,-243.0
0,2020-07-15,World,13352870.0,13299163.0,578176.0,578319.0,53707.0,-143.0
0,2020-07-15,United Kingdom,291107.0,291373.0,44915.0,44968.0,-266.0,-53.0
0,2020-07-15,Germany,199235.0,199726.0,9074.0,9071.0,-491.0,3.0
0,2020-07-15,Iran,262964.0,262173.0,13200.0,13211.0,791.0,-11.0
0,2020-07-15,Turkey,214966.0,214993.0,5400.0,5402.0,-27.0,-2.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")